# Retrieve Data from Database

In order to connect to the database, first, make sure:
1. …you have a local (running) database with the `hostname`, `database` name, `username`, and `port` number as specified in `/resources/config/gadb_pg_config.py`.
2. …the `airports` and `flights_and_weather` tables have data.

If the `airports` and `flights_and_weather` tables do not have data, you must first run `/download_and_prepare_datasets/02_prepare_and_store_data.ipynb` to populate the database.

In [1]:
import os
import time

In [2]:
os.chdir(os.path.join('..','resources'))

## Connect to Database

In [3]:
# Database configuration details
from config import gadb_pg_config as cfg

# To connect to SQL database
import sqlalchemy as db
# from sqlalchemy import create_engine, MetaData, Table, text, types

# To enter passwords without exposing them
from getpass import getpass

Most of the database information is already in `cfg` (above). However, you will have to enter your password below.

In [4]:
password = getpass('Enter database password')

Enter database password········


In [5]:
start_time = time.time()

In [6]:
db_string = f"postgresql+psycopg2://{cfg.username}:{password}@{cfg.hostname}:{cfg.port}/{cfg.database}"

In [7]:
engine = db.create_engine(
    future=True,
#     echo=True,
    url=db_string
)

## Access Database Tables and Create Table Aliases

In [8]:
db_meta = db.MetaData()

In [9]:
# Access the `airports` and `flights_and_weather` tables and assign them to variables
airports_table = db.Table('airports', db_meta, autoload_with=engine)
faw_table = db.Table('flights_and_weather', db_meta, autoload_with=engine)
faw = faw_table.alias('faw')
orig = airports_table.alias('orig')
dest = airports_table.alias('dest')

## Extract Full, Joined Dataset to Pandas Dataframe

In [10]:
from io import StringIO
import pandas as pd

In [11]:
# Function to extract from database query to Pandas dataframe
# Adapted from <https://towardsdatascience.com/optimizing-pandas-read-sql-for-postgres-f31cd7f707ab>
def read_sql_inmem(query, db_engine, **kwargs):
    copy_sql = "COPY ({query}) TO STDOUT WITH CSV {head}".format(
       query=query, head="HEADER"
    )
    conn = db_engine.raw_connection()
    cur = conn.cursor()
    store = StringIO()
    cur.copy_expert(copy_sql, store)
    store.seek(0)
    df = pd.read_csv(store, **kwargs)
    return df

In [12]:
# The query composed as an SQLAlchemy table object
get_all = (
    db.select([
        faw,
        orig.c.lat_decimal.label('origin_lat'),
        orig.c.lon_decimal.label('origin_lon'),
        dest.c.lat_decimal.label('destination_lat'),
        dest.c.lon_decimal.label('destination_lon')
    ])
    .select_from(
        faw
        .join(orig, orig.c.iata_code == faw.c.origin_airport, isouter=True)
        .join(dest, dest.c.iata_code == faw.c.destination_airport, isouter=True)
    )
    .order_by(faw.c.id)
)

In [13]:
def print_shape(df):
    df_rows, df_cols = df.shape
    print(f"{df_rows:,} rows × {df_cols:,} columns")

In [14]:
def df_details(df):
    return pd.concat(
        [
            df.dtypes,
            df.isna().sum(),
            pd.Series(
                data=[df[col].dropna().is_unique for col in df.columns],
                index=df.columns
            )
        ],
        axis=1,
        keys=['data_type','null_count','unique']
    )

In [15]:
# Run the query, and save the result to a dataframe
combined_df = read_sql_inmem(
    query=get_all,
#     query=get_all.limit(10).compile(engine, compile_kwargs={"literal_binds": True}),
    db_engine=engine,
    index_col='id',
    converters={'cancelled':(lambda x: True if x == 't' else False)}
)
# .sort_index(kind='mergesort')

print_shape(combined_df)

5,468,069 rows × 35 columns


In [16]:
with pd.option_context('display.max_columns',None):
    display(combined_df.head())

,carrier_code,flight_number,origin_airport,destination_airport,flight_date,scheduled_elapsed_time,tail_number,departure_delay,arrival_delay,delay_carrier,delay_weather,delay_national_aviation_system,delay_security,delay_late_aircarft_arrival,cancelled,scheduled_departure_dt,scheduled_arrival_dt,actual_departure_dt,actual_arrival_dt,station_x,hourlydrybulbtemperature_x,hourlyprecipitation_x,hourlystationpressure_x,hourlyvisibility_x,hourlywindspeed_x,station_y,hourlydrybulbtemperature_y,hourlyprecipitation_y,hourlystationpressure_y,hourlyvisibility_y,hourlywindspeed_y,origin_lat,origin_lon,destination_lat,destination_lon
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,AS,121,SEA,ANC,2019-07-01,215,N611AS,0,-16,0,0,0,0,0,False,2019-07-01 00:40:00,2019-07-01 03:15:00,2019-07-01 00:40:00,2019-07-01 02:59:00,7.279302e+10,65.0,0.0,29.60,10.0,6.0,7.027253e+10,57.0,0.0,30.04,10.0,3.0,47.449,-122.309,61.174,-149.996
1,B6,1092,BQN,FLL,2019-07-01,155,N589JB,-19,-40,0,0,0,0,0,False,2019-07-01 01:26:00,2019-07-01 04:01:00,2019-07-01 01:07:00,2019-07-01 03:21:00,7.851401e+10,79.0,NaN,29.81,10.0,9.0,7.478301e+10,78.0,0.0,30.01,10.0,0.0,18.495,-67.129,26.072,-80.153
2,B6,668,PSE,MCO,2019-07-01,175,N662JB,73,69,0,0,0,0,69,False,2019-07-01 01:15:00,2019-07-01 04:10:00,2019-07-01 02:28:00,2019-07-01 05:19:00,NaN,NaN,NaN,NaN,NaN,NaN,7.220501e+10,76.0,0.0,29.93,10.0,3.0,18.008,-66.563,28.429,-81.316
3,F9,402,LAX,DEN,2019-07-01,143,N706FR,-1,-7,0,0,0,0,0,False,2019-07-01 00:59:00,2019-07-01 04:22:00,2019-07-01 00:58:00,2019-07-01 04:15:00,7.229502e+10,65.0,0.0,29.60,10.0,6.0,7.256500e+10,62.0,0.0,24.69,10.0,8.0,33.942,-118.408,39.858,-104.667
4,F9,790,PDX,DEN,2019-07-01,156,N350FR,21,10,0,0,0,0,0,False,2019-07-01 00:55:00,2019-07-01 04:31:00,2019-07-01 01:16:00,2019-07-01 04:41:00,7.269802e+10,66.0,0.0,29.92,10.0,6.0,7.256500e+10,62.0,0.0,24.70,8.0,10.0,45.589,-122.597,39.858,-104.667


In [17]:
df_details(combined_df)

,data_type,null_count,unique
carrier_code,object,0,False
flight_number,int64,0,False
origin_airport,object,0,False
destination_airport,object,0,False
flight_date,object,0,False
scheduled_elapsed_time,int64,0,False
tail_number,object,6973,False
departure_delay,int64,0,False
arrival_delay,int64,0,False
delay_carrier,int64,0,False


In [18]:
print(f'Elapsed time: {(time.time() - start_time):,.3f} seconds.')

Elapsed time: 382.805 seconds.


In [ ]:
from path import Path
import numpy as np
import pandas as pd

In [ ]:
# Read in the data
# Note: The following data has been normalized between 0 and 1
data = Path('../Resources/loans.csv')
df = pd.read_csv(data)
df.head()

In [ ]:
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

In [ ]:
# Separate the features (X) from the target (y)
y = df["cancelled"]
X = df.drop(columns="cancelled")

In [ ]:
# Use the train_test_split function to create training and testing subsets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
   y,  random_state=1, stratify=y)
X_train.shape

In [ ]:
# Create Logistic Regression Model
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs', random_state=1)
classifier

In [ ]:
# Fit(Train) the data
classifier.fit(X_train, y_train)

In [ ]:
# Make predictions using the test data
y_pred = model.predict(X_test)
results = pd.DataFrame({
    "Prediction": y_pred, 
    "Cancelled": y_test
}).reset_index(drop=True)
results.head()

In [ ]:
# Validate using test data
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

In [ ]:
# Generate confusion_matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))